### Time Series of Wikipedia Traffic

This notebook takes a sample of the time series web traffic, makes some predictions with some simple time series models, and then takes one of them to make predictions on all 145,000 wikipedia sites.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import timeit
start_time = timeit.default_timer()
train = pd.read_csv("train_2.csv")
elapsed = timeit.default_timer() - start_time
print("Time to load data: ", round(elapsed, 2), "s")
print("Shape of Data: ", train.shape)

Time to load data:  16.59 s
Shape of Data:  (145063, 804)


In [2]:
train.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2017-09-01,2017-09-02,2017-09-03,2017-09-04,2017-09-05,2017-09-06,2017-09-07,2017-09-08,2017-09-09,2017-09-10
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,19.0,33.0,33.0,18.0,16.0,27.0,29.0,23.0,54.0,38.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,32.0,30.0,11.0,19.0,54.0,25.0,26.0,23.0,13.0,81.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,6.0,6.0,7.0,2.0,4.0,7.0,3.0,4.0,7.0,6.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,7.0,19.0,19.0,9.0,6.0,16.0,19.0,30.0,38.0,4.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.0,16.0,19.0,9.0,20.0,23.0,28.0,14.0,8.0,7.0


In order to run the time series I need the data to be transposed to show 1 article per column:

In [3]:
train = train.transpose()
train.head()

,0,1,2,3,4,5,6,7,8,9,...,145053,145054,145055,145056,145057,145058,145059,145060,145061,145062
Page,2NE1_zh.wikipedia.org_all-access_spider,2PM_zh.wikipedia.org_all-access_spider,3C_zh.wikipedia.org_all-access_spider,4minute_zh.wikipedia.org_all-access_spider,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,5566_zh.wikipedia.org_all-access_spider,91Days_zh.wikipedia.org_all-access_spider,A'N'D_zh.wikipedia.org_all-access_spider,AKB48_zh.wikipedia.org_all-access_spider,ASCII_zh.wikipedia.org_all-access_spider,...,Drake_(músico)_es.wikipedia.org_all-access_spider,Skam_(serie_de_televisión)_es.wikipedia.org_al...,Legión_(serie_de_televisión)_es.wikipedia.org_...,Doble_tentación_es.wikipedia.org_all-access_sp...,Mi_adorable_maldición_es.wikipedia.org_all-acc...,Underworld_(serie_de_películas)_es.wikipedia.o...,Resident_Evil:_Capítulo_Final_es.wikipedia.org...,Enamorándome_de_Ramón_es.wikipedia.org_all-acc...,Hasta_el_último_hombre_es.wikipedia.org_all-ac...,Francisco_el_matemático_(serie_de_televisión_d...
2015-07-01,18,11,1,35,NaN,12,NaN,118,5,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-02,11,14,0,13,NaN,7,NaN,26,23,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-03,5,15,1,10,NaN,4,NaN,30,14,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-04,13,18,1,94,NaN,5,NaN,24,12,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Turn the first row to a header:

In [4]:
new_header = train.iloc[0]
train = train[1:]
train.columns = new_header
train.head(3)

Page,2NE1_zh.wikipedia.org_all-access_spider,2PM_zh.wikipedia.org_all-access_spider,3C_zh.wikipedia.org_all-access_spider,4minute_zh.wikipedia.org_all-access_spider,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,5566_zh.wikipedia.org_all-access_spider,91Days_zh.wikipedia.org_all-access_spider,A'N'D_zh.wikipedia.org_all-access_spider,AKB48_zh.wikipedia.org_all-access_spider,ASCII_zh.wikipedia.org_all-access_spider,...,Drake_(músico)_es.wikipedia.org_all-access_spider,Skam_(serie_de_televisión)_es.wikipedia.org_all-access_spider,Legión_(serie_de_televisión)_es.wikipedia.org_all-access_spider,Doble_tentación_es.wikipedia.org_all-access_spider,Mi_adorable_maldición_es.wikipedia.org_all-access_spider,Underworld_(serie_de_películas)_es.wikipedia.org_all-access_spider,Resident_Evil:_Capítulo_Final_es.wikipedia.org_all-access_spider,Enamorándome_de_Ramón_es.wikipedia.org_all-access_spider,Hasta_el_último_hombre_es.wikipedia.org_all-access_spider,Francisco_el_matemático_(serie_de_televisión_de_2017)_es.wikipedia.org_all-access_spider
2015-07-01,18,11,1,35,NaN,12,NaN,118,5,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-02,11,14,0,13,NaN,7,NaN,26,23,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-03,5,15,1,10,NaN,4,NaN,30,14,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Check the average number of missing values for each website:

In [5]:
train.isna().sum().mean()

48.443421134265805

Each page has an average of 48 missing rows. This is a lot. I'm going to do simple forwards (and backwards) fill imputation.

In [6]:
train = train.fillna(method = "ffill")
train.head(2)

Page,2NE1_zh.wikipedia.org_all-access_spider,2PM_zh.wikipedia.org_all-access_spider,3C_zh.wikipedia.org_all-access_spider,4minute_zh.wikipedia.org_all-access_spider,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,5566_zh.wikipedia.org_all-access_spider,91Days_zh.wikipedia.org_all-access_spider,A'N'D_zh.wikipedia.org_all-access_spider,AKB48_zh.wikipedia.org_all-access_spider,ASCII_zh.wikipedia.org_all-access_spider,...,Drake_(músico)_es.wikipedia.org_all-access_spider,Skam_(serie_de_televisión)_es.wikipedia.org_all-access_spider,Legión_(serie_de_televisión)_es.wikipedia.org_all-access_spider,Doble_tentación_es.wikipedia.org_all-access_spider,Mi_adorable_maldición_es.wikipedia.org_all-access_spider,Underworld_(serie_de_películas)_es.wikipedia.org_all-access_spider,Resident_Evil:_Capítulo_Final_es.wikipedia.org_all-access_spider,Enamorándome_de_Ramón_es.wikipedia.org_all-access_spider,Hasta_el_último_hombre_es.wikipedia.org_all-access_spider,Francisco_el_matemático_(serie_de_televisión_de_2017)_es.wikipedia.org_all-access_spider
2015-07-01,18.0,11.0,1.0,35.0,NaN,12.0,NaN,118.0,5.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-02,11.0,14.0,0.0,13.0,NaN,7.0,NaN,26.0,23.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
train = train.fillna(method = "bfill")

In [8]:
train.head(2)

Page,2NE1_zh.wikipedia.org_all-access_spider,2PM_zh.wikipedia.org_all-access_spider,3C_zh.wikipedia.org_all-access_spider,4minute_zh.wikipedia.org_all-access_spider,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,5566_zh.wikipedia.org_all-access_spider,91Days_zh.wikipedia.org_all-access_spider,A'N'D_zh.wikipedia.org_all-access_spider,AKB48_zh.wikipedia.org_all-access_spider,ASCII_zh.wikipedia.org_all-access_spider,...,Drake_(músico)_es.wikipedia.org_all-access_spider,Skam_(serie_de_televisión)_es.wikipedia.org_all-access_spider,Legión_(serie_de_televisión)_es.wikipedia.org_all-access_spider,Doble_tentación_es.wikipedia.org_all-access_spider,Mi_adorable_maldición_es.wikipedia.org_all-access_spider,Underworld_(serie_de_películas)_es.wikipedia.org_all-access_spider,Resident_Evil:_Capítulo_Final_es.wikipedia.org_all-access_spider,Enamorándome_de_Ramón_es.wikipedia.org_all-access_spider,Hasta_el_último_hombre_es.wikipedia.org_all-access_spider,Francisco_el_matemático_(serie_de_televisión_de_2017)_es.wikipedia.org_all-access_spider
2015-07-01,18.0,11.0,1.0,35.0,38.0,12.0,61.0,118.0,5.0,6.0,...,14.0,26.0,50.0,11.0,21.0,13.0,13.0,51.0,16.0,77.0
2015-07-02,11.0,14.0,0.0,13.0,38.0,7.0,61.0,26.0,23.0,3.0,...,14.0,26.0,50.0,11.0,21.0,13.0,13.0,51.0,16.0,77.0


In [9]:
train.isna().sum().mean()

0.0

To decide which model is best, I'll use a small sample of 100 websites:

In [10]:
sample = train.sample(n = 100, axis = 1)
sample.head(2)

Page,Handball-Europameisterschaft_2016_de.wikipedia.org_all-access_spider,Heribert_Sasse_de.wikipedia.org_all-access_spider,Go!プリンセスプリキュア_ja.wikipedia.org_desktop_all-agents,Category:Nude_crawling_women_commons.wikimedia.org_all-access_spider,Royal_Rumble_(2017)_es.wikipedia.org_mobile-web_all-agents,Auschwitz_fr.wikipedia.org_all-access_spider,Шехзаде_Мустафа_ru.wikipedia.org_all-access_spider,Newton_Knight_en.wikipedia.org_mobile-web_all-agents,UFC_on_Fox:_Maia_vs._Condit_en.wikipedia.org_desktop_all-agents,Catégorie:Sigle_de_3_caractères_fr.wikipedia.org_all-access_all-agents,...,白雪公主与猎人_zh.wikipedia.org_all-access_all-agents,Mika_(chanteur)_fr.wikipedia.org_desktop_all-agents,Alexandra_Lacrabère_fr.wikipedia.org_all-access_spider,File:Debbie_Does_Dallas.ogv_commons.wikimedia.org_mobile-web_all-agents,中田久美_ja.wikipedia.org_all-access_all-agents,火影忍者疾風傳_zh.wikipedia.org_all-access_all-agents,Fahrenheit_451_en.wikipedia.org_desktop_all-agents,List_of_countries_by_GDP_(nominal)_en.wikipedia.org_desktop_all-agents,File:Hognose_playing_dead.webm_commons.wikimedia.org_desktop_all-agents,Amoklauf_von_Winnenden_de.wikipedia.org_desktop_all-agents
2015-07-01,1.0,1.0,1439.0,2.0,41.0,26.0,4.0,113.0,486.0,489.0,...,69.0,882.0,2.0,100.0,270.0,970.0,1407.0,9103.0,15.0,180.0
2015-07-02,3.0,5.0,1235.0,1.0,41.0,23.0,5.0,96.0,486.0,512.0,...,55.0,727.0,2.0,100.0,231.0,1300.0,1317.0,8002.0,11.0,207.0


In [11]:
sample.shape

(803, 100)

Now withold 1 month for testing:

In [12]:
sample_test = sample.tail(30)
sample = sample.head(803 - 30)

In [13]:
sample.tail(2)

Page,Handball-Europameisterschaft_2016_de.wikipedia.org_all-access_spider,Heribert_Sasse_de.wikipedia.org_all-access_spider,Go!プリンセスプリキュア_ja.wikipedia.org_desktop_all-agents,Category:Nude_crawling_women_commons.wikimedia.org_all-access_spider,Royal_Rumble_(2017)_es.wikipedia.org_mobile-web_all-agents,Auschwitz_fr.wikipedia.org_all-access_spider,Шехзаде_Мустафа_ru.wikipedia.org_all-access_spider,Newton_Knight_en.wikipedia.org_mobile-web_all-agents,UFC_on_Fox:_Maia_vs._Condit_en.wikipedia.org_desktop_all-agents,Catégorie:Sigle_de_3_caractères_fr.wikipedia.org_all-access_all-agents,...,白雪公主与猎人_zh.wikipedia.org_all-access_all-agents,Mika_(chanteur)_fr.wikipedia.org_desktop_all-agents,Alexandra_Lacrabère_fr.wikipedia.org_all-access_spider,File:Debbie_Does_Dallas.ogv_commons.wikimedia.org_mobile-web_all-agents,中田久美_ja.wikipedia.org_all-access_all-agents,火影忍者疾風傳_zh.wikipedia.org_all-access_all-agents,Fahrenheit_451_en.wikipedia.org_desktop_all-agents,List_of_countries_by_GDP_(nominal)_en.wikipedia.org_desktop_all-agents,File:Hognose_playing_dead.webm_commons.wikimedia.org_desktop_all-agents,Amoklauf_von_Winnenden_de.wikipedia.org_desktop_all-agents
2017-08-10,4.0,0.0,253.0,3.0,68.0,23.0,8.0,775.0,92.0,1238.0,...,57.0,204.0,1.0,285.0,268.0,460.0,1648.0,6470.0,1.0,91.0
2017-08-11,2.0,1.0,200.0,0.0,44.0,11.0,12.0,822.0,63.0,5181.0,...,38.0,223.0,1.0,295.0,271.0,525.0,1382.0,6165.0,1.0,106.0


In [14]:
sample_test.tail()

Page,Handball-Europameisterschaft_2016_de.wikipedia.org_all-access_spider,Heribert_Sasse_de.wikipedia.org_all-access_spider,Go!プリンセスプリキュア_ja.wikipedia.org_desktop_all-agents,Category:Nude_crawling_women_commons.wikimedia.org_all-access_spider,Royal_Rumble_(2017)_es.wikipedia.org_mobile-web_all-agents,Auschwitz_fr.wikipedia.org_all-access_spider,Шехзаде_Мустафа_ru.wikipedia.org_all-access_spider,Newton_Knight_en.wikipedia.org_mobile-web_all-agents,UFC_on_Fox:_Maia_vs._Condit_en.wikipedia.org_desktop_all-agents,Catégorie:Sigle_de_3_caractères_fr.wikipedia.org_all-access_all-agents,...,白雪公主与猎人_zh.wikipedia.org_all-access_all-agents,Mika_(chanteur)_fr.wikipedia.org_desktop_all-agents,Alexandra_Lacrabère_fr.wikipedia.org_all-access_spider,File:Debbie_Does_Dallas.ogv_commons.wikimedia.org_mobile-web_all-agents,中田久美_ja.wikipedia.org_all-access_all-agents,火影忍者疾風傳_zh.wikipedia.org_all-access_all-agents,Fahrenheit_451_en.wikipedia.org_desktop_all-agents,List_of_countries_by_GDP_(nominal)_en.wikipedia.org_desktop_all-agents,File:Hognose_playing_dead.webm_commons.wikimedia.org_desktop_all-agents,Amoklauf_von_Winnenden_de.wikipedia.org_desktop_all-agents
2017-09-06,3.0,3.0,185.0,3.0,56.0,32.0,3.0,666.0,87.0,610.0,...,47.0,246.0,0.0,258.0,45354.0,779.0,1826.0,7972.0,5.0,114.0
2017-09-07,3.0,6.0,247.0,0.0,37.0,47.0,6.0,776.0,80.0,691.0,...,40.0,237.0,4.0,233.0,8686.0,819.0,1651.0,10351.0,3.0,97.0
2017-09-08,2.0,5.0,168.0,0.0,50.0,87.0,13.0,942.0,88.0,538.0,...,47.0,211.0,2.0,297.0,63059.0,829.0,1574.0,6094.0,4.0,113.0
2017-09-09,2.0,7.0,202.0,4.0,55.0,170.0,7.0,822.0,65.0,519.0,...,70.0,187.0,2.0,311.0,72225.0,869.0,984.0,4364.0,5.0,82.0
2017-09-10,1.0,5.0,241.0,4.0,67.0,72.0,9.0,1364.0,98.0,621.0,...,55.0,195.0,2.0,299.0,42704.0,901.0,1120.0,4866.0,2.0,74.0


The data sets match. Now I try 8 different time series models and make predictions with the test set, one after the other. I'm going to measure the time and compare the RMSE to the test set, before choosing one to predict on all the data.

In [15]:
import warnings
warnings.filterwarnings("ignore")

# 1. Autoregression (AR)
from statsmodels.tsa.ar_model import AR

preds_data = pd.DataFrame()
start_date = "2017-08-12"
end_date = "2017-09-10"

# Measure time
start_time = timeit.default_timer()

# Fit model
for column in sample:
    model = AR(sample[column])
    model_fit = model.fit()
# Make prediction
    yhat = model_fit.predict(start_date, end_date)
    preds_data[column] = yhat

# End time
elapsed = timeit.default_timer() - start_time

print("Time for 100 predictions: ", round(elapsed, 2), "s")
#print("RMSE: ", (((preds_data - sample_test) ** 2).mean() ** 0.5).mean())


C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\

C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\

Time for 100 predictions:  1.32 s


C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\

In [16]:
# 2. Moving Average (MA)
from statsmodels.tsa.arima_model import ARMA

preds_data = pd.DataFrame()
start_date = "2017-08-12"
end_date = "2017-09-10"

# Measure time
start_time = timeit.default_timer()

# Fit model
for column in sample:
    model = ARMA(sample[column], order = (0,1))
    model_fit = model.fit()
# Make prediction
    yhat = model_fit.predict(start_date, end_date)
    preds_data[column] = yhat

# End time
elapsed = timeit.default_timer() - start_time

print("Time for 100 predictions: ", round(elapsed, 2), "s")
print("RMSE: ", (((preds_data - sample_test) ** 2).mean() ** 0.5).mean())

C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\

C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\

C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\david\

Time for 100 predictions:  26.28 s
RMSE:  628.5883237010505


In [17]:
import warnings
warnings.filterwarnings("ignore")

# 3. Autoregressive Moving Average (ARMA)
from statsmodels.tsa.arima_model import ARMA

preds_data = pd.DataFrame()
start_date = "2017-08-12"
end_date = "2017-09-10"

# Measure time
start_time = timeit.default_timer()

# Fit model
for column in sample:
    model = ARMA(sample[column], order = (1,0))
    model_fit = model.fit()
# Make prediction
    yhat = model_fit.predict(start_date, end_date)
    preds_data[column] = yhat

# End time
elapsed = timeit.default_timer() - start_time

print("Time for 100 predictions: ", round(elapsed, 2), "s")
print("RMSE: ", (((preds_data - sample_test) ** 2).mean() ** 0.5).mean())

Time for 100 predictions:  23.54 s
RMSE:  619.1067867110179


In [18]:
# 4. Autoregressive Integrated Moving Average (ARIMA)
from statsmodels.tsa.arima_model import ARIMA

preds_data = pd.DataFrame()
start_date = "2017-08-12"
end_date = "2017-09-10"

# Measure time
start_time = timeit.default_timer()

# Fit model
for column in sample:
    model = ARIMA(sample[column], order = (1, 0, 0))
    model_fit = model.fit()
# Make prediction
    yhat = model_fit.predict(start_date, end_date)
    preds_data[column] = yhat

# End time
elapsed = timeit.default_timer() - start_time

print("Time for 100 predictions: ", round(elapsed, 2), "s")
print("RMSE: ", (((preds_data - sample_test) ** 2).mean() ** 0.5).mean())

Time for 100 predictions:  11.47 s
RMSE:  619.1067867110179


In [19]:
# 5. SARIMAX
from statsmodels.tsa.statespace.sarimax import SARIMAX

preds_data = pd.DataFrame()
start_date = "2017-08-12"
end_date = "2017-09-10"

# Measure time
start_time = timeit.default_timer()

# Fit model
for column in sample:
    model = SARIMAX(sample[column])
    model_fit = model.fit()
# Make prediction
    yhat = model_fit.forecast(steps = 30)
    preds_data[column] = yhat

# End time
elapsed = timeit.default_timer() - start_time

print("Time for 100 predictions: ", round(elapsed, 2), "s")
print("RMSE: ", (((preds_data - sample_test) ** 2).mean() ** 0.5).mean())

Time for 100 predictions:  5.95 s
RMSE:  715.367489695995


In [20]:
# 6. SARIMAX parameters
from statsmodels.tsa.statespace.sarimax import SARIMAX

preds_data = pd.DataFrame()
start_date = "2017-08-12"
end_date = "2017-09-10"

# Measure time
start_time = timeit.default_timer()

# Fit model
for column in sample:
    model = SARIMAX(sample[column], order = (1,1,0))
    model_fit = model.fit()
# Make prediction
    yhat = model_fit.forecast(steps = 30)
    preds_data[column] = yhat

# End time
elapsed = timeit.default_timer() - start_time

print("Time for 100 predictions: ", round(elapsed, 2), "s")
print("RMSE: ", (((preds_data - sample_test) ** 2).mean() ** 0.5).mean())

Time for 100 predictions:  5.56 s
RMSE:  552.2099410411818


In [21]:
# 7. Simple Exponential Smoothing (SES)
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

preds_data = pd.DataFrame()
start_date = "2017-08-12"
end_date = "2017-09-10"

# Measure time
start_time = timeit.default_timer()

# Fit model
for column in sample:
    model = SimpleExpSmoothing(sample[column])
    model_fit = model.fit()
# Make prediction
    yhat = model_fit.predict(start_date, end_date)
    preds_data[column] = yhat

# End time
elapsed = timeit.default_timer() - start_time

print("Time for 100 predictions: ", round(elapsed, 2), "s")
print("RMSE: ", (((preds_data - sample_test) ** 2).mean() ** 0.5).mean())

Time for 100 predictions:  3.42 s
RMSE:  532.6294859335399


In [22]:
# 8. Holt Winters Exponential Smoothing (HWES)
from statsmodels.tsa.holtwinters import ExponentialSmoothing

preds_data = pd.DataFrame()
start_date = "2017-08-12"
end_date = "2017-09-10"

# Measure time
start_time = timeit.default_timer()

# Fit model
for column in sample:
    model = ExponentialSmoothing(sample[column])
    model_fit = model.fit()
# Make prediction
    yhat = model_fit.predict(start_date, end_date)
    preds_data[column] = yhat

# End time
elapsed = timeit.default_timer() - start_time

print("Time for 100 predictions: ", round(elapsed, 2), "s")
print("RMSE: ", (((preds_data - sample_test) ** 2).mean() ** 0.5).mean())

Time for 100 predictions:  2.7 s
RMSE:  532.6294859335399


Given the time taken as well as the RMSE, the best approach could simply be to do an autoregression:

In [23]:
train.shape

(803, 145063)

In [24]:
#sample = train.iloc[:,0:80000]
#sample.shape

There are a few observations which have just 1 observation. The Autoregression won't work with these columns. I need to remove them before running the model on all the data.

In [25]:
nunique = train.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
sample = train.drop(cols_to_drop, axis=1)

In [26]:
# Autoregression (AR)
warnings.filterwarnings("ignore")

preds_data = pd.DataFrame()
start_date = "2017-09-11"
end_date = "2017-11-13"

# Measure time
start_time = timeit.default_timer()

# Fit model
for column in sample:
#    print(column, "start")
    model = AR(sample[column])
    model_fit = model.fit()
# Make prediction
    yhat = model_fit.predict(start_date, end_date)
    preds_data[column] = yhat
#    print(column, "done")
    sample.columns.get_loc(column)

# End time
elapsed = timeit.default_timer() - start_time

print("Time for All predictions: ", round(elapsed, 2), "s")


Time for All predictions:  4040.7 s


In [27]:
# preds_data.to_pickle("full_preds.pkl")
#preds_data = pd.read_pickle("full_preds.pkl")
preds_data.shape

(64, 145027)

Now I have the predictions, I just need to do a bit of data wrangling and use the key to create the submission. FIrst, add the columns I took away.

In [28]:
extra_cols = train[cols_to_drop]
extra_cols = extra_cols.iloc[0:64,:]
extra_cols.index = preds_data.index
preds_data = pd.concat([preds_data, extra_cols], axis=1)
preds_data.shape

(64, 145063)

In [29]:
preds_data.head(2)

,2NE1_zh.wikipedia.org_all-access_spider,2PM_zh.wikipedia.org_all-access_spider,3C_zh.wikipedia.org_all-access_spider,4minute_zh.wikipedia.org_all-access_spider,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,5566_zh.wikipedia.org_all-access_spider,91Days_zh.wikipedia.org_all-access_spider,A'N'D_zh.wikipedia.org_all-access_spider,AKB48_zh.wikipedia.org_all-access_spider,ASCII_zh.wikipedia.org_all-access_spider,...,xss_commons.wikimedia.org_all-access_all-agents,xss_de.wikipedia.org_all-access_spider,Artificial_Insemination's_Art_www.mediawiki.org_all-access_spider,Special:WhatLinksHere/File:DSO-Ensemble.jpg_www.mediawiki.org_all-access_spider,Thread:Extension_talk:Phpbb_Single_Sign-On/_www.mediawiki.org_all-access_spider,User_talk:199.15.233.174_www.mediawiki.org_all-access_spider,xss_ja.wikipedia.org_desktop_all-agents,????:Andrey_Belloly_1.jpg_ru.wikipedia.org_all-access_spider,xss_ru.wikipedia.org_all-access_spider,xss_ja.wikipedia.org_all-access_spider
2017-09-11,30.398645,29.278639,5.338112,20.097365,16.646249,18.450573,4.428612,43.077505,45.871557,26.727910,...,43289.0,0.0,0.0,0.0,0.0,0.0,138722.0,0.0,0.0,0.0
2017-09-12,25.961315,29.629339,5.484612,20.236384,18.629958,19.298680,13.698189,52.749437,45.555685,26.784241,...,43289.0,0.0,0.0,0.0,0.0,0.0,138722.0,0.0,0.0,0.0


Firstly, turn the data in the long form used in the submission

In [30]:
unstacked = pd.DataFrame(preds_data.unstack())
unstacked.shape

(9284032, 1)

In [31]:
unstacked.head(2)

0
2NE1_zh.wikipedia.org_all-access_spider 2017-09-11  30.398645
                                        2017-09-12  25.961315

Now read in the final version of the key data:

In [32]:
keys = pd.read_csv("key_2.csv")

In [33]:
keys.tail(4)

,Page,Id
8993902,龙生九子_zh.wikipedia.org_mobile-web_all-agents_20...,33aecaf259d1
8993903,龙生九子_zh.wikipedia.org_mobile-web_all-agents_20...,a36228b64466
8993904,龙生九子_zh.wikipedia.org_mobile-web_all-agents_20...,26887d3e5c99
8993905,龙生九子_zh.wikipedia.org_mobile-web_all-agents_20...,06ad06979f80


In [34]:
keys.shape

(8993906, 2)

Now turn the index to a column:

In [35]:
unstacked["key"] = unstacked.index

In [36]:
unstacked.head(2)

0  \
2NE1_zh.wikipedia.org_all-access_spider 2017-09-11  30.398645   
                                        2017-09-12  25.961315   

                                                                                                  key  
2NE1_zh.wikipedia.org_all-access_spider 2017-09-11  (2NE1_zh.wikipedia.org_all-access_spider, 2017...  
                                        2017-09-12  (2NE1_zh.wikipedia.org_all-access_spider, 2017...

And split the data on the comma and space

In [37]:
unstacked["key"] = unstacked["key"].astype(str)
unstacked[["name", "date"]] = unstacked['key'].str.split(', ',n = 1, expand=True)

In [38]:
unstacked.head(2)

0  \
2NE1_zh.wikipedia.org_all-access_spider 2017-09-11  30.398645   
                                        2017-09-12  25.961315   

                                                                                                  key  \
2NE1_zh.wikipedia.org_all-access_spider 2017-09-11  ('2NE1_zh.wikipedia.org_all-access_spider', Ti...   
                                        2017-09-12  ('2NE1_zh.wikipedia.org_all-access_spider', Ti...   

                                                                                          name  \
2NE1_zh.wikipedia.org_all-access_spider 2017-09-11  ('2NE1_zh.wikipedia.org_all-access_spider'   
                                        2017-09-12  ('2NE1_zh.wikipedia.org_all-access_spider'   

                                                                                           date  
2NE1_zh.wikipedia.org_all-access_spider 2017-09-11  Timestamp('2017-09-11 00:00:00', freq='D'))  
                                        2017-09-12  Timestamp('2017-09-12 00:00:00', freq='D'))

Now trim off the edges:

In [39]:
unstacked["name"] = unstacked["name"].str[2:-1]
unstacked["date"] = unstacked["date"].str[11:-22]

In [40]:
unstacked.head(2)

0  \
2NE1_zh.wikipedia.org_all-access_spider 2017-09-11  30.398645   
                                        2017-09-12  25.961315   

                                                                                                  key  \
2NE1_zh.wikipedia.org_all-access_spider 2017-09-11  ('2NE1_zh.wikipedia.org_all-access_spider', Ti...   
                                        2017-09-12  ('2NE1_zh.wikipedia.org_all-access_spider', Ti...   

                                                                                       name  \
2NE1_zh.wikipedia.org_all-access_spider 2017-09-11  2NE1_zh.wikipedia.org_all-access_spider   
                                        2017-09-12  2NE1_zh.wikipedia.org_all-access_spider   

                                                          date  
2NE1_zh.wikipedia.org_all-access_spider 2017-09-11  2017-09-11  
                                        2017-09-12  2017-09-12

And concat them together using a hyphen:

In [41]:
unstacked["joining"] = unstacked["name"].map(str) + "_" + unstacked["date"].map(str)

Merge the 2 datasets togther:

In [44]:
final = pd.merge(keys, unstacked, how = "left", left_on = "Page", right_on = "joining")

Finally, select only the columns I need and rename them:

In [45]:
final = final.iloc[:,0:3]
final.columns = ["Page", "Id", "Views"]
final = final[["Id", "Views"]]
final.head(2)

,Id,Views
0,0b293039387a,587.635642
1,7114389dd824,694.114726


There are some missing somehow. I will predict 0 and also turn the negative predictions to 0 and round to intgers.

In [47]:
import numpy as np
final['Views'] = np.where(final['Views'] >= 0, final['Views'], 0)

In [49]:
final["Views"] = round(final["Views"],0)

And turn into a csv for submission:

In [50]:
# final.to_csv("final_sub.csv", index = False)